
this is data anlysis 

a typical programmer (data scientist) would do these things

we all hear terms like data science, big data, and AI.  Here is a brief view into what that actually looks like

... I built Buckaroo to enahnce those workflows, here it is

In order for me to explain buckaroo I have to give some background



# Explaining Buckarooto my non-technical friends 

This presentation walks through how datascientists use Buckaroo to enhance their workflows.

## The tools that Datascientists commonly use.

Buckaroo enhances the tools that data practitionrs already use.

### Python

The simple syntax of the python programming language makes it easy to learn.  This presentation has snippets of python code.

### NumPy and Pandas

NumPy is the matrix math library for python, it speeds up matrix operations 25-100x.  Pandas is built on top of NumPy and it enables idiomatic manipulation of timeseries and other data commonly found in the wild.

In [ ]:
a = 5
b = 10
a+b

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
rng = np.random.default_rng(1234)
# Generate data
x = rng.uniform(0, 10, size=100)
y = x + rng.normal(size=100)

# Initialize layout
fig, ax = plt.subplots(figsize = (9, 9))

# Add scatterplot
ax.scatter(x, y, s=60, alpha=0.7, edgecolors="k")

# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
b, a = np.polyfit(x, y, deg=1)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(0, 10, num=100)

# Plot regression line
ax.plot(xseq, a + b * xseq, color="k", lw=2.5);

# The Jupyter notebook

You are seeing this presentation in the Jupyter notebook.  The jupyter programming environment was created by grad students to be used interactively and to leave an artifiact that combines narrative text, analysis code, and graphics.  The interactivity is key to analyzing and manipulating data.

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/paddy/code/example-notebooks/citibike-trips.csv") 
df

In [ ]:
# Once you have a dataset like this there are a lot of operations you might want to perform
df['tripduration'].mean()

In [ ]:
df['start station name'].value_counts()

In [ ]:
df.groupby('start station name').mean('tripduration')

In [ ]:
df['tripduration'].hist()

In [ ]:
df[(df['tripduration'] > df['tripduration'].quantile(.01)) & (df['tripduration'] < df['tripduration'].quantile(.99))]['tripduration'].hist()

# Why I wrote Buckaroo

Thank you for bearing with me this far.  You now have seen the PyData ecosystem and a small sample of how it is used.
These are all powerful tools, but a bit cumbersome to use.  I look at multiple different datasets a day, and I want to quickly understand them.  I don't want to type a bunch of commands to get the overview I'm looking for and I want to be able to look at the raw data.  Here is Buckaroo

In [ ]:
from buckaroo.buckaroo_widget import BuckarooWidget
df = pd.read_csv("/Users/paddy/code/example-notebooks/citibike-trips.csv") 
df

In [ ]:
df['starttime'].is_monotonic_increasing, df['start station latitude'].is_monotonic_decreasing

In [ ]:
df['start station latitude'].is_monotonic_increasing, df['start station latitude'].is_monotonic_decreasing

In [ ]:
ser = df['starttime']
(ser.is_monotonic_increasing or ser.is_monotonic_decreasing)

In [ ]:
BuckarooWidget(
    df,
    column_config_overrides={
        'tripduration': {'color_map_config': {
        'color_rule': 'color_map',
        'map_name': 'BLUE_TO_YELLOW'}}})

In [ ]:
from buckaroo.pluggable_analysis_framework.pluggable_analysis_framework import ColAnalysis
from buckaroo.dataflow.dataflow import StylingAnalysis
class MonotonicStat(StylingAnalysis):
    provides_defaults = dict(is_monotonic=False)
    
    @staticmethod
    def series_summary(sampled_ser, ser):
        #print("here", ser.name, len(ser), ser.is_monotonic_increasing, ser.is_monotonic_decreasing)
        return dict(is_monotonic=(ser.is_monotonic_increasing or ser.is_monotonic_decreasing))
    pinned_rows = [
        { 'primary_key_val': 'is_monotonic', 'displayer_args': {'displayer': 'obj'}}]
    df_display_name = "is_monotonic"
    data_key = "empty"
    
w = BuckarooWidget(df[:40_000])
w.add_analysis(MonotonicStat)
w

In [ ]:
w = BuckarooWidget(pd.DataFrame({'a':[1,2,3], 'b':[3,2,1], 'c':[10,20,30], 'd':[10, 25, 30], 'e':[10, 5, 30]}))
w.add_analysis(MonotonicStat)
w

In [ ]:
df[df['start station latitude'] <df['start station latitude'].quantile(.01) ]

# Explaining Buckarooto my non-technical friends 

This presentation show my side project Buckaroo and the tools it is built on top of.

## Background on python and how it's used in datascience

Before we dive into exciting visuals and interactive programs, let’s lay some groundwork.

Python is a popular programming language, you might also have heard of java, C, and javascript.  Datascientists have come to rely on python because it balances speed of execution (C is faster) with ease of use and learning.

### NumPy

Matrix math is at the heart of linear regression, image recognition, and AI like ChatGPT. NumPy was written by Travis Oliphant in 2006 to accelerate matrix operations in python.   In many cases NumPy accelerates matrix operations to 25-100x faster than raw python.

Here is NumPy and the Matplotlib charting

# Pandas

In 2011 financial quant researcher Wes Mckinney released pandas which made analysis of realworld data easier, timeseries data in particular.  Pandas was built on top of NumPy, and allowed computations to be run on mixed datasets (you could have a set of temperature observations ordered by time of day, with a string column for location).

Pandas took what was already technically possible and increased the usability so more (work|concepts|manipulations) could be expressed in a natural way.

The following code shows reading a 300,000 line csv file about citibike trips

# The Jupyter notebook

I have been demonstrating the entire PyData ecosystem inside of the Jupyter notebook.  This is an interactive analysis and documentation environment built around python.  While a grad student Fernando Perez wanted a better [interactive environment](https://en.wikipedia.org/wiki/IPython) for playing with data in 2001.  In 2011 the IPython first released the [jupyter notebook](https://en.wikipedia.org/wiki/Project_Jupyter) interface you see here.

Combining small snippets of analysis code, with charts, and narrative text allowed academics to write and share research in ways that were cumbersome before.
(Maybe show emacs/vscode traditional method of writing code).  This is particularly important for data intensive analysis.  You need to look at the data and play with it iteratively.  This interface works very well for the problem that data scientists and academics deal with every day.